In [4]:
import os
import cv2
from ultralytics import YOLO
import supervision as sv

# Step 1: Train the YOLO Model
def train_yolo():
    # Initialize the YOLO model with pretrained weights
    model = YOLO("yolov8n.pt")  #...use m or l  
    # Train the model with your custom dataset
    model.train(
        data="data.yaml",  # path to your data configuration file
        epochs=10,             # number of training epochs
        imgsz=640,              # input image size
        batch=16,               # batch size
        name="yolov8n-greyhound"  # name of the training run
    )

# Step 2: Process Video using the Trained YOLO Model
def process_video(video_path, output_path, model_path):
    # Load the fine-tuned YOLO model
    model = YOLO(model_path)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Verify the output directory and permissions
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)

    if output_dir and not os.access(output_dir, os.W_OK):
        raise PermissionError(f"Write permission denied for the directory {output_dir}")

    assert cap.isOpened(), "Error reading video file"

    # Get video properties
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

    # Initialize VideoWriter with a successful FourCC code
    fourcc_code = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(output_path, fourcc_code, fps, (w, h))

    if not video_writer.isOpened():
        raise IOError(f"Error initializing video writer with path {output_path}")

    # Custom function to filter greyhound detections
    def filter_greyhound_detections(detections):
        greyhound_class_id = 0  
        mask = detections.class_id == greyhound_class_id
        filtered_detections = sv.Detections(
            xyxy=detections.xyxy[mask],
            confidence=detections.confidence[mask],
            class_id=detections.class_id[mask]
        )
        return filtered_detections

    # Process video frames
    box_annotator = sv.BoxAnnotator(thickness=4)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Video frame is empty or video processing has been successfully completed.")
            break

        # Perform object detection
        results = model(frame, imgsz=1280)

        # Extract bounding boxes, confidences, and class IDs
        boxes = results[0].boxes.xyxy.cpu().numpy()
        confidences = results[0].boxes.conf.cpu().numpy()
        class_ids = results[0].boxes.cls.cpu().numpy().astype(int)

        # Create detections object
        detections = sv.Detections(
            xyxy=boxes,
            confidence=confidences,
            class_id=class_ids
        )

        # Filter greyhound detections
        greyhound_detections = filter_greyhound_detections(detections)

        # Annotate the frame
        annotated_frame = box_annotator.annotate(scene=frame, detections=greyhound_detections)

        # Write the processed frame to the output video
        video_writer.write(annotated_frame)

    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()

    print(f"Processed video saved to {output_path}")

# Train the model (uncomment the line below to train)
train_yolo()

# Path to your greyhound racing video
GREYHOUND_RACING_VIDEO_PATH = "greyhound_1.mp4"
# Output path for the processed video
OUTPUT_VIDEO_PATH = "greyhound_racing_output.mp4"
# Path to the fine-tuned model weights
MODEL_PATH = "runs/detect/yolov8n-greyhound10/weights/best.pt"

# Process the video using the trained model
process_video(GREYHOUND_RACING_VIDEO_PATH, OUTPUT_VIDEO_PATH, MODEL_PATH)


100%|██████████| 49.7M/49.7M [00:04<00:00, 10.8MB/s]


Ultralytics YOLOv8.2.70  Python-3.8.19 torch-2.4.0 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolov8m.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n-greyhound10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

train: Scanning C:\Users\mohit\Downloads\datasets\train\labels.cache... 504 images, 3 backgrounds, 0 corrupt: 100%|██████████| 504/504 [00:00<?, ?it/s]

WARNING  Box and segment counts should be equal, but got len(segments) = 2235, len(boxes) = 2265. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.



val: Scanning C:\Users\mohit\Downloads\datasets\valid\labels.cache... 48 images, 0 backgrounds, 0 corrupt: 100%|██████████| 48/48 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolov8n-greyhound10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000909, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolov8n-greyhound10
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20      7.15G      2.088      3.606      2.154        149        640:  56%|█████▋    | 18/32 [02:11<01:42,  7.32s/it]


KeyboardInterrupt: 